Import the required packages


In [ ]:
!pip install langchain-google-genai langchain-google-vertexai langchain-google-community langchain-core --quiet
from google.cloud import aiplatform, bigquery
from vertexai.language_models import TextEmbeddingModel
from langchain_google_vertexai import VertexAIEmbeddings

Configure the project with Env

In [ ]:
project = "qwiklabs-gcp-02-4c9c7fb5e8ec"
location = "US"
dataset = "Aurora_dataset"
table_name = "AuroraFaqs"
table_embed = "Faqs_Embedded"
gcs_uri = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

Create the BigQuery Client


In [ ]:
bq_client = bigquery.Client(project=project)

Creating the Dataset

In [ ]:
def create_dataset():
    dataset_id = f"{bq_client.project}.{dataset}"
    dataset_ref = bigquery.Dataset(dataset_id)
    dataset_ref.location = location  # ✅ FIXED
    try:
        bq_client.create_dataset(dataset_ref, exists_ok=True)
        print("✅ Dataset created.")
    except Exception as e:
        print("✅ Dataset may already exist or error:", e)

Load the data into the BigQuery

In [ ]:
def load_csv_to_bigquery():
  try:
    # Create the table
    table_id = "{}.{}.{}".format(bq_client.project, dataset, table_name)
    table = bigquery.Table(table_id)
    destination_table = bq_client.get_table(table_id)
    table = bq_client.create_table(table)

    job_config = bigquery.LoadJobConfig(
        schema=[
        bigquery.SchemaField("question", "STRING"),
        bigquery.SchemaField("answer", "STRING"),
        ],
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
    )

    load_job = bq_client.load_table_from_uri(
        gcs_uri, table_id, job_config=job_config
    )
    load_job.result()
    print("✅ Data loaded into table")
  except Exception as e:
    print("✅ Table may already exist or error:", e)

load_csv_to_bigquery()


In [ ]:
!bq show --location=us --connection --project_id={project} "embedding_conn"

Establish the connection to the BigQuery Dataset we created

In [ ]:
connection_service_account = "bqcx-569779670169-huh6@gcp-sa-bigquery-condel.iam.gserviceaccount.com" # @param {"type": "string"}
connection_member = f"serviceAccount:{connection_service_account}"

Add the permissions for the service account

In [ ]:
!gcloud projects add-iam-policy-binding {project} --member='serviceAccount:bqcx-568234179561-n0wm@gcp-sa-bigquery-condel.iam.gserviceaccount.com' --role='roles/aiplatform.user' --condition=None --quiet


Create Embedding model

In [ ]:
query = f"""
CREATE OR REPLACE MODEL `{project}.{dataset}.Embeddings`
REMOTE WITH CONNECTION `US.embedding_conn` OPTIONS (ENDPOINT = 'text-embedding-005');
"""

query_job = bq_client.query(query)
query_job.result()



Generate embeddings

In [ ]:
query = f"""
CREATE OR REPLACE TABLE `{project}.{dataset}.faqs_embedded` AS SELECT *
FROM ML.GENERATE_EMBEDDING(
    MODEL `{project}.{dataset}.Embeddings`,
(SELECT CONCAT(question, ' ', answer) content FROM `{project}.{dataset}.AuroraFaqs`)
);
"""


query_job = bq_client.query(query)
query_job.result()


In [ ]:

query_insert = f"""
INSERT INTO `{project}.{dataset}.faqs_embedded` (content, ml_generate_embedding_result)
SELECT content, ml_generate_embedding_result
FROM ML.GENERATE_EMBEDDING(
    MODEL `{project}.{dataset}.Embeddings`,
    (SELECT CONCAT(question, ' ', answer) AS content FROM `{project}.{dataset}.AuroraFaqs`)
);
"""

query_job = bq_client.query(query_insert)
query_job.result()

Load Vector Search Results from BigQuery Table

In [ ]:
embedding = VertexAIEmbeddings(
    model_name="text-embedding-005", project=project
)

from langchain_google_community import BigQueryVectorStore

store = BigQueryVectorStore(
    project_id=project,
    dataset_name=dataset,
    table_name=table_embed,
    location=location,
    embedding=embedding,
    embedding_field="ml_generate_embedding_result",
)


In [ ]:
def fetch_results(user_question):
    query = f"""
    SELECT
      query.query,
      result.base.question,
      result.base.answer,
      result.distance
    FROM VECTOR_SEARCH(
      TABLE `{project}.{dataset}.{table_name}`,
      'embedding',
      (
        SELECT
          ml_generate_embedding_result AS embedding,
          '{user_question}' AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `{project}.{dataset}.{table_embed}`,
          (SELECT '{user_question}' AS content)
        )
      ),
      top_k => 3,
      options => '{{"fraction_lists_to_search": 1.0}}'
    ) AS result
    """
    return bq_client.query(query).to_dataframe()


In [ ]:
TABLE_ID = f"{project}.{dataset}.{table_embed}"
EMBED_MODEL_ID = f"{project}.{dataset}.faqs_embedded"
def fetch_results(user_question):
    query = f"""
    SELECT
      query.query,
      result.base.question,
      result.base.answer,
      result.distance
    FROM VECTOR_SEARCH(
      TABLE `{TABLE_ID}`,
      'embedding',
      (
        SELECT
          ml_generate_embedding_result AS embedding,
          '{user_question}' AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `{EMBED_MODEL_ID}`,
          (SELECT '{user_question}' AS content)
        )
      ),
      top_k => 3,
      options => '{{"fraction_lists_to_search": 1.0}}'
    ) AS result
    """
    return bq_client.query(query).to_dataframe()

In [ ]:
from vertexai.preview.generative_models import GenerativeModel
chat_model = GenerativeModel("gemini-2.0-flash-001")

def generate_bot_response(user_input):
    results = fetch_results(user_input)
    context = "\n\n".join([f"Q: {row['question']}\nA: {row['answer']}" for _, row in results.iterrows()])
    prompt = f"You are a helpful assistant for the town of Aurora Bay. Use the following FAQ context to answer:\n\n{context}\n\nUser: {user_input}"
    response = chat_model.generate_content(prompt)
    return response.text
while True:
    question = input("User: ")
    if question.strip().lower() in ["exit", "no", "quit"]:
        print("\nSession ended.")
        break
    print("\nAssistant:", generate_bot_response(question))